# 생물정보학 및 실습 1 - Term Project용 Mission 3 환경 셋업

```
생물정보학 및 실습 1
서울대학교 협동과정 생물정보학전공 2022년 1학기

이 노트북은 Creative Commons BY-SA 조건으로 재사용 가능합니다.
장혜식 <hyeshik@snu.ac.kr>, 서울대학교 생명과학부
```

이 노트북은 Google Colab을 이용해서 Term Project를 할 수 있도록 환경을 준비해 줍니다. 터미널 사용이 원활하지 않은 상태이고, 데이터를 오랫동안 보관하기 어렵기 때문에 되도록 개인용 컴퓨터에서 [Anaconda](https://www.anaconda.com/)나 [JupyterLab](https://jupyterlab.readthedocs.io/en/stable/) 등을 설치하고 사용하시는 것이 편하지만, 개인용 컴퓨터 사양이 부족한 경우에는 Google CoLab을 사용할 수도 있습니다. 개인용 컴퓨터를 쓸 때는 Windows에서는 [WSL2](https://docs.microsoft.com/en-us/windows/wsl/install-win10)를 쓰시면 리눅스 커맨드라인 환경을 쓸 수 있습니다. Google Colab은 창을 닫은 후 90분 후, 또는 창을 처음 열고 12시간 후에 환경을 닫아버리기 때문에 매번 실습할 때마다 새로 셋업이 필요합니다. `Runtime->Run All`을 실행해서 전체를 한 번 실행하시면 준비됩니다. 모든 것이 잘 안 되고 뭔가 꼬인 느낌이 강하게 들 때는 `Runtime->Factory reset runtime`을 누르고 새로 시작하시면 될 수도 있습니다.


## 시퀀싱 데이터 다운로드
이제 실습 수업에서 필요한 시퀀싱 데이터를 다운로드 받아서 설치합시다. 드라이브 공간만 충분하면 데이터는 계속 저장돼 있기 때문에 학기 전체를 통해서 딱 한 번만 하시면 됩니다. Colab에서는 작업용 데이터를 Google Drive 폴더 아래의 `binfo-datapack1`에 저장합니다. 실습용 데이터는 `/content/drive/MyDrive/binfo1-datapack1`에 풀리게 됩니다.

### (Option 2) CoLab외의 다른 환경에서 접속한 경우
본인 컴퓨터나 연구실 워크스테이션 등에서 작업하는 경우에는 다음 명령어로 다운로드 받으시면 됩니다. 첫 번째에는 wget이 접속오류를 내는 경우가 많으니, 접속에러가 발생하면 다시 한 번 실행해 보세요. `/content/drive/Mydrive` 부분은 원하는 저장 경로로 바꿔서 지정하시면 됩니다.

In [ ]:
!wget -O - --no-check-certificate https://hyeshik.qbio.io/binfo/binfo1-datapack1.tar | tar -C /content/drive/MyDrive -xf -

### 파일 체크섬 확인

받은 데이터의 MD5 체크섬을 확인합니다. 한참 걸립니다. 각 파일들의 원래 체크섬은 다음과 같습니다. 한 번 비교해 보세요.

```
140aaf30bcb9276cc716f8699f04ddd6  CLIP-35L33G.bam
f1b3336ed7e2f97d562dcc71641251bd  CLIP-35L33G.bam.bai
328883a73d507eafbf5b60bd6b906201  RNA-control.bam
02073818e2f398a73c3b76e5169de1ca  RNA-control.bam.bai
b09550d09d6c2a4ce27f0226f426fdb1  RNA-siLin28a.bam
fef112c727244060ea62d3f2564a07f6  RNA-siLin28a.bam.bai
28bbd0c47d725669340c784f1b772c01  RNA-siLuc.bam
43590fdc4d81905c0432e0d1cb8cfd5b  RNA-siLuc.bam.bai
5c08a9297307bc83259e658c4474f0cc  RPF-siLin28a.bam
a1bb3e29be412dfd7fd8d16b1b1acc4c  RPF-siLin28a.bam.bai
f2eebf50943024d0116c9cd3e744c707  RPF-siLuc.bam
dc24f69e8f571fc8be30f28ce5b84fcd  RPF-siLuc.bam.bai
```

In [ ]:
!md5sum drive/MyDrive/binfo1-datapack1/*

실습에 필요한 프로그램들 몇 가지를 추가로 설치합니다.

In [ ]:
!conda install -y bedtools bioawk

# Free Project

본 노트북에서는 레퍼런스 논문의 Figure 2E를 재현한다. 이 그림은 LIN28A와 상호작용 하는 서열 주위의 Watson-Crick (WC) pair co-occurence의 상대 빈도를 나타낸다. Binding motif는 논문의 앞선 부분(Figure 2A-D)에서 제안한  

LIN28A과 잘 알려진 target인 Pre-let-7이 상호작용할 때,  binding motif가 언제나 stem 근처의 terminal loop에 위치해 있으며, pre-let-7의 stem이 binding을 돕는다는 데이터가 있다. 따라서 binding motif를 새롭게 발견하면서 이런 현상이 유지되는지 확인하고자 그 주위의 2nd structure를 분석하는 것이 이 figure의 목적이 된다.

## Process
Figure 2E는 두 부분으로 구성되어 있다.
1. WebLogo를 이용하여 나타낸 normalized base probability
2. 

In [2]:
%cd ./binfo1-datapack1/

/home/pmkyu98/Lecture/BioInfo 1/binfo1-datapack1


### Data Processing

먼저 논문과 같이 read depth>=50 조건으로 filtering 하기 위해서 CLIP-35L33G.bam 파일을 pileup 한다.

In [3]:
!samtools mpileup CLIP-35L33G.bam > CLIP-35L33G.pileup

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


In [7]:
!head CLIP-35L33G.pileup

chr1	3056473	N	4	^!G^!G^!G^!G	HGGI
chr1	3056474	N	4	AAAA	HB@I
chr1	3056475	N	4	AAAA	GG?I
chr1	3056476	N	4	TTTT	HGGH
chr1	3056477	N	4	GGGG	HEGI
chr1	3056478	N	4	GGGG	GEEI
chr1	3056479	N	4	AAAA	EA=I
chr1	3056480	N	4	AAAA	@AEH
chr1	3056481	N	4	GGGG	GFFI
chr1	3056482	N	4	TTTT	G=BG


In [6]:
!awk '$4 >= 50 { print $0; }' CLIP-35L33G.pileup > filtered-CLIP-35L33G.pileup

In [3]:
!head filtered-CLIP-35L33G.pileup

chr1	3222722	N	124	CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A	HCGGHEIHHHIBDDIIIGIG>GIHGDHHDDIAEIEEBIGIBGG8GIH@HEIHDIHDIIGHFFGFIIHI@HIGHIIIIIIIIGGBIBHHIHDGIIHBIF=GDDHIHGIHDHIHGBGBFGGHHGGF
chr1	3222723	N	146	GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G^!G	E>HIHFIIHHIGDFIII<IHBIIHGDHEB?I@HIEIBIII;>G;GII@@HIHGIIEIIGIH0DIIIHIDHIBGIIIIIIIIIF?IBHHIHDGIIHGIBGEHDEI1HGHHEHFIGB@GH>GHDE>IGHIBHIEGGHGBGHGIGIDII
chr1	3222724	N	155	AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A^!A

Pandas 이용해 정리. match와 substituion만 남김

In [5]:
import pandas as pd
import re

pileup = pd.read_csv('filtered-CLIP-35L33G.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])

In [23]:
toremove = re.compile('[<>$*#]') # 이 부분을 잘 모르겠음
pileup['matches'] = pileup['basereads'].apply(lambda x: toremove.sub('', x))

In [52]:
pileup

,chrom,pos,_ref,count,basereads,quals,matches,CRES
0,chr1,3222722,N,124,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,HCGGHEIHHHIBDDIIIGIG>GIHGDHHDDIAEIEEBIGIBGG8GI...,CCCAAAAAAAAAAAAAAAAAAAAAAAAA^!A^!A^!A^!A^!A^!A...,0.164379
1,chr1,3222723,N,146,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,E>HIHFIIHHIGDFIII<IHBIIHGDHEB?I@HIEIBIII;>G;GI...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,-0.000000
2,chr1,3222724,N,155,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,H>GIGHHIHHDG:@IIIBIHDIIHG=HHDDG;IHHH;IIIDG?DIE...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,-0.000000
3,chr1,3222725,N,157,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,HGDEHHIIHHIG?DIIIEIGDIIHDBBH=DI;IIHBBIII4GD3GI...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,0.203572
4,chr1,3222726,N,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,HB:IHGIIGGIGDFHII@IFBHGHGBHGCBI@IIHGBIGI;>E@BI...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,1.401164
...,...,...,...,...,...,...,...,...
25210334,MU069435.1,1564,N,305,>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,DIDHGHIIGIGIEDIGIHIIG?FBGG@E=:A7G8@HHIHIIDDBII...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,-0.000000
25210335,MU069435.1,1565,N,297,>>>>AAAAAAAAAAAA$AA$AA$A$AA$AA$A$AAAAAAAAAAAAA...,DIDHBHHIIIGICGIGIEIHDEFBGE8C=1?1G88IHIHIIDGBHI...,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,-0.000000
25210336,MU069435.1,1566,N,289,>>>>CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,DIDHG>IIHIHHCEIIIEB5D@;=H5BIHIHHI<DAIIBDIIFIIH...,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,0.033667
25210337,MU069435.1,1567,N,291,>>>>T$T$T$T$T$T$T$T$T$T$T$T$T$T$T$TTTTTTTTTT$T...,DIDHFHGIIIIIDDIII?B=G::B=G=DIHIHIID@BIIGGIIHIH...,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,-0.000000


Shannon entropy 계산하여 CRES>0.8 기준으로 filtering.
Shannon entropy 계산식은 레퍼런스 논문의 Supplementary part를 참조하였다.

In [43]:
from collections import Counter
import numpy as np

def CRES(seq):
    letters = Counter(seq)
    bases = ['A', 'T', 'G', 'C']
    
    shannon = 0
    whole = letters['A'] + letters['T'] + letters['G'] + letters['C']
    for base in bases:
        if base in letters:
            p_n = letters[base] / whole
            shannon += (p_n * np.log2(p_n))
        
    return -1 * shannon

In [45]:
pileup['CRES'] = pileup['matches'].apply(lambda x: CRES(x))

In [53]:
data = pileup[pileup.CRES >= 0.8].copy()

In [54]:
data

,chrom,pos,_ref,count,basereads,quals,matches,CRES
4,chr1,3222726,N,156,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,HB:IHGIIGGIGDFHII@IFBHGHGBHGCBI@IIHGBIGI;>E@BI...,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,1.401164
46,chr1,3224576,N,168,CCCG*GGGG*GGGGC*GGGGGGGGGG**GGGGGCTCTGCCCCCCCG...,BGI?H?G8HIEGHIIIBIHFIE9GIIG?=E?:HHGBA8GCB??0GH...,CCCGGGGGGGGGCGGGGGGGGGGGGGGGCTCTGCCCCCCCGCCCGG...,1.261116
318,chr1,4631665,N,170,ggggggggggggggGGGCGGCGCGGGGAGGAGAAGAAGAGGAAAAA...,GDIHIDIIIIIDEHGDHBIHGIHHBIBCH4DHII?IG?IIGI8HII...,ggggggggggggggGGGCGGCGCGGGGAGGAGAAGAAGAGGAAAAA...,1.071389
348,chr1,4631695,N,511,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,HBDDDD<IHGEIIEIACBIFHBIIIHBIHHHAIDDBBHIHIIHEG:...,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,0.834922
376,chr1,4631745,N,78,CCCCCCCTTTCCCTCTCTTCTCCTCTTTTcccccccCCCCCCCCCC...,8HIHHBIF7HIIFFIIIBB:GEHIBIH;IEG>GHII?IIGIBGGGG...,CCCCCCCTTTCCCTCTCTTCTCCTCTTTTcccccccCCCCCCCCCC...,0.834648
...,...,...,...,...,...,...,...,...
25210020,MU069435.1,505,N,2074,TCCTCTTTCTCCTTCCTCCCCCCCCCCCCCCTCCCTCC*CCCCCTC...,@@:<B??>@@D@F@73?HIIH5I>DIIBEHHGBDI;C=7E9IHGG>...,TCCTCTTTCTCCTTCCTCCCCCCCCCCCCCCTCCCTCCCCCCCTCC...,0.822535
25210096,MU069435.1,581,N,2854,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,H?FEC@GBBADEEBEBD?E?>88@?G@BIE@>DB;GEBFGGECEB....,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,1.314127
25210097,MU069435.1,582,N,2858,GGGGGGGGGGGGGGGGGGGGGGGGGGGTTTGGGGGGGGGTGGTGGG...,DGHB>@EBB>BCEAEBFAC?C18D9D@@@BEBHEEHG;EAACAB0H...,GGGGGGGGGGGGGGGGGGGGGGGGGGGTTTGGGGGGGGGTGGTGGG...,0.952813
25210240,MU069435.1,1023,N,86,>>>>GGGGGGAGGG$AAGGAGGGGGGAGGGGGGGAGAGAGGGAGGC...,DIDHD/BB?FBIDE<EF?@FIHGHGFEHGIG@IGBEIGIEA@D>F=...,GGGGGGAGGGAAGGAGGGGGGAGGGGGGGAGAGAGGGAGGCGAGGT...,0.935560
